In [1]:
using DataFrames, CSV, DelimitedFiles, JuMP, Gurobi
const GRB_ENV = Gurobi.Env()

Academic license - for non-commercial use only


Gurobi.Env(Ptr{Nothing} @0x000000007c006e20, false, 0)

In [2]:
NUM_CREWS = 10                
BREAK_LENGTH = 2       # how long at base to be considered "rested"

# tradeoffs
BETA = 100             # cost of one area unit burned / cost of mile traveled
ALPHA = 200            # cost of crew-day of suppression / cost of mile traveled
LINE_PER_CREW = 17     # how much perimeter prevented per crew per time period

FIRE_CODE = 1
BASE_CODE = 2

2

In [3]:
struct GlobalData
    
    ff_dist::Matrix{Float64}
    bf_dist::Matrix{Float64}
    ff_tau::Matrix{Int64}
    bf_tau::Matrix{Int64}
    
end

struct CrewStatus
    
    rest_by::Vector{Int64}
    current_fire::Vector{Int64}
    rested_periods::Vector{Int64}
    
end

struct RegionData
    
    crew_regions::Vector{Int64}
    fire_regions::Vector{Int64}
    
end

mutable struct RouteData
    
    routes_per_crew::Vector{Int64} # could add in length
    route_costs::Matrix{Float64}
    fires_fought::BitArray{4}
    out_of_reg::BitArray{3}
    
end

mutable struct SuppressionPlanData
    
    plans_per_fire::Vector{Int64} # could add in length
    plan_costs::Matrix{Float64}
    crews_present::Array{Int8, 3}
    
end

In [4]:
function get_rotation_orders(crew_regions)
    
    # initialize output
    out = Dict()
    
    # get the unique regions where there are crews
    regions = unique(crew_regions)
    
    # for each region
    for region in regions
        
        # initialize dictionary corresponding to the order
        out[region] = Dict() 
        crews_in_region = 0
        
        # for each crew in the region
        for crew in 1:NUM_CREWS
            
            if crew_regions[crew] == region
                
                # update crew count, log rotation order 
                crews_in_region += 1
                out[region][crew] = crews_in_region
            end
        end
    end
    
    return out
end

get_rotation_orders (generic function with 1 method)

In [5]:
# crew, from_type, from_ix, to_type, to_ix, from_time, to_time, from_rested, to_rested, exited_region

In [6]:
function arc_exits_region(crew, from_type, from_ix, to_type, to_ix, region_data)
    
    # get the region where the arc originates
    from_region = 0
    if from_type == FIRE_CODE
        from_region = region_data.fire_regions[from_ix]
    elseif from_type == BASE_CODE
        from_region = region_data.crew_regions[from_ix]
    else
        throw(DomainError(from_type, "from_type invalid"))
    end
    
    # get the region where the arc terminates
    to_region = 0
    if to_type == FIRE_CODE
        to_region = region_data.fire_regions[to_ix]
    elseif to_type == BASE_CODE
        to_region = region_data.crew_regions[to_ix]
    else
        throw(DomainError(from_type, "to_type invalid"))
    end
    
    # if these are different regions
    if from_region != to_region
        
        # if the crew is leaving its home region
        if region_data.crew_regions[crew] == from_region
        
            # return the region that the arc exited
            return from_region
        
        end
        
    end
    
    # otherwise
    return 0
    
end     

arc_exits_region (generic function with 1 method)

In [850]:
function generate_arcs(gd, rd, cs)
    
    # get fire-to-fire arcs
    ff = [[c, FIRE_CODE, f_from, FIRE_CODE, f_to, t_from, t_from + gd.ff_tau[f_to, f_from], rest, rest]
          for c=1:NUM_CREWS, f_from=1:NUM_FIRES, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    ff = copy(reduce(hcat, ff)')

    # get fire-to-fire arcs from start, based on cs.current crew locations
    from_start_ff = [[c, FIRE_CODE, cs.current_fire[c], FIRE_CODE, f_to, 0, gd.ff_tau[f_to, cs.current_fire[c]], 0, 0]
                      for c=1:NUM_CREWS, f_to=1:NUM_FIRES if cs.current_fire[c] != -1]
    from_start_ff = copy(reduce(hcat, from_start_ff)')

    # get base-to-fire arcs
    rf = [[c, BASE_CODE, c, FIRE_CODE, f_to, t_from, t_from + gd.bf_tau[c, f_to], rest, rest]
           for c=1:NUM_CREWS, f_to=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    rf = copy(reduce(hcat, rf)')

    # get base-to-fire arcs from start
    from_start_rf = [[c, BASE_CODE, c, FIRE_CODE, f_to, 0, gd.bf_tau[c, f_to], 0, 0]
                      for c=1:NUM_CREWS, f_to=1:NUM_FIRES if cs.current_fire[c] == -1]
    from_start_rf = copy(reduce(hcat, from_start_rf)')

    # get fire-to-base arcs
    fr = [[c, FIRE_CODE, f_from, BASE_CODE, c, t_from, t_from + gd.bf_tau[c, f_from], rest, rest]
           for c=1:NUM_CREWS, f_from=1:NUM_FIRES, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    fr = copy(reduce(hcat, fr)')

    # get fire-to-base arcs from start, based on cs.current crew locations
    from_start_fr = [[c, FIRE_CODE, cs.current_fire[c], BASE_CODE, c, 0, gd.bf_tau[c, cs.current_fire[c]], 0, 0]
                      for c=1:NUM_CREWS if cs.current_fire[c] != -1]
    from_start_fr = copy(reduce(hcat, from_start_fr)')

    # get base-to-base arcs
    rr = [[c, BASE_CODE, c, BASE_CODE, c, t_from, t_from + 1 + (BREAK_LENGTH - 1) * rest, 0, rest]
          for c=1:NUM_CREWS, t_from=1:NUM_TIME_PERIODS, rest=0:1]
    rr = copy(reduce(hcat, rr)')
    rr_rested = [[c, BASE_CODE, c, BASE_CODE, c, t_from, t_from + 1, 1, 1]
          for c=1:NUM_CREWS, t_from=1:NUM_TIME_PERIODS]
    rr_rested  = copy(reduce(hcat, rr_rested)')

    # get base-to-base arcs from start, based on cs.current days rested
    from_start_rr = [[c, BASE_CODE, c, BASE_CODE, c, 0, 
                      1 + (BREAK_LENGTH - max(cs.rested_periods[c], 0) - 1) * rest, 0, rest] 
                      for c=1:NUM_CREWS, rest=0:1 if cs.current_fire[c] == -1]
    from_start_rr = copy(reduce(hcat, from_start_rr)')

    A = vcat(ff, from_start_ff, rf, from_start_rf, fr, from_start_fr, rr, rr_rested, from_start_rr)

    out_of_region = [arc_exits_region(A[i, 1], A[i, 2], A[i, 3], A[i, 4], A[i, 5], rd) 
                     for i in 1:length(A[:, 1])]
    A = hcat(A, out_of_region)
    
    return A
end

generate_arcs (generic function with 1 method)

In [8]:
function get_distance(from_type, from_ix, to_type, to_ix, fire_fire, base_fire)
    
    dist = 0
    
    # if fire to fire
    if (from_type == FIRE_CODE) & (to_type == FIRE_CODE)
        dist = fire_fire[from_ix, to_ix]
    
    # if fire to base
    elseif (from_type == FIRE_CODE) & (to_type == BASE_CODE)
        dist = base_fire[to_ix, from_ix]
    
    # if base to fire
    elseif (from_type == BASE_CODE) & (to_type == FIRE_CODE)
        dist = base_fire[from_ix, to_ix]
        
    # otherwise dist still 0
    end
    
    return dist
end 

get_distance (generic function with 1 method)

In [120]:
function get_arc_costs(gd, arcs, cost_param_dict)
    
    # get number of arcs
    n_arcs = length(arcs[:, 1])
    
    # initialize costs to 0
    costs = zeros(n_arcs)
    
    # if there is travel cost per mile
    if "cost_per_mile" in keys(cost_param_dict)
        
        # find the miles for each arc
        miles_per_arc =  [get_distance(arcs[i, 2], arcs[i, 3], 
                                       arcs[i, 4], arcs[i, 5], 
                                       gd.ff_dist, gd.bf_dist) for i in 1:n_arcs]
        # add to costs
        costs = costs .+ (cost_param_dict["cost_per_mile"] * miles_per_arc)
    end
    
    # if there are rest violations
    if "rest_violation" in keys(cost_param_dict)
        
        # find the rest violation scores
        rest_violation_matrix = cost_param_dict["rest_violation"]
        rest_violations = [(arcs[i, 8] == 0) & (arcs[i, 6] > 0) ? 
                           rest_violation_matrix[arcs[i, 1], arcs[i, 6]] : 0
                           for i in 1:n_arcs]
        
        # add to costs
        costs = costs .+ rest_violations
    end
    
    if "fight_fire" in keys(cost_param_dict)
        costs = costs .+ [(arcs[i, 4] == FIRE_CODE) ? cost_param_dict["fight_fire"] : 0
                          for i in 1:n_arcs]
    end
    
    # if we have to adjust for linking dual constraints
    if "linking_dual" in keys(cost_param_dict)
        
        # get the dual variables
        rho = cost_param_dict["linking_dual"]
        
        # get linking costs (really benefits) if arc goes to a fire
        linking_costs = [((arcs[i, 4] == FIRE_CODE) & (arcs[i, 7] <= NUM_TIME_PERIODS)) ? 
                          - rho[arcs[i, 5], arcs[i, 7]] : 0
                          for i in 1:n_arcs]
        
        # add to costs
        costs = costs .+ linking_costs
        
    end
    
    # if we have to adjust for linking dual constraints
    if "out_of_region_dual" in keys(cost_param_dict)
        
        # get needed regional info
        regs = cost_param_dict["region_data"].crew_regions
        rot_order = cost_param_dict["rotation_order"]
        
        # get the dual variables
        eta = cost_param_dict["out_of_region_dual"]

        # get adjustment for crew allotment
        c1 = [(arcs[i, 10] > 0) ? sum(eta[arcs[i, 1], t_0]
                                        for t_0=arcs[i, 6]:NUM_TIME_PERIODS
                                      ) : 0
                                                   
               for i in 1:n_arcs
             ]
        
        # get adjustment for region average allotment
        c2 = [(arcs[i, 10] > 0) ? sum(eta[c, t_0]
                                            for c in keys(rot_order[regs[arcs[i, 1]]]),
                                                t_0=arcs[i, 6]:NUM_TIME_PERIODS) /
                                        length(keys(rot_order[regs[arcs[i, 1]]])) : 0
                                                   
               for i in 1:n_arcs
             ]
        
        # get adjustment for big-M constraint
        c3 = [(arcs[i, 10] > 0) ? NUM_TIME_PERIODS * eta[arcs[i, 1], arcs[i, 6]] : 0
               for i in 1:n_arcs
             ]
            
        # add to costs
        costs = costs .+ c1 .- c2 .+ c3
        
    end   
    
    return costs
end

get_arc_costs (generic function with 1 method)

In [10]:
function positive(x)
    
    if x > 0
        return 1
    end
    
    return 0
end

function is_one(x)
    
    if x == 1
        return 1
    end
    
    return 0
end

is_one (generic function with 1 method)

In [11]:
# should return matrix indexed by crew, time, 
function get_rest_penalties(rest_by_periods, lambda, accounting_func)
    
    penalties = zeros(NUM_CREWS, NUM_TIME_PERIODS)
    
    for c in 1:NUM_CREWS
        penalties[c, :] = [lambda * accounting_func(t - rest_by_periods[c]) 
                           for t in 1:NUM_TIME_PERIODS]
    end
    
    return penalties    
end

get_rest_penalties (generic function with 1 method)

In [12]:
struct KeyArcIndices
    
    # fire flow data
    f_out::Array{Vector{Int64}}
    f_in::Array{Vector{Int64}}
    
    # base flow data
    b_out::Array{Vector{Int64}}
    b_in::Array{Vector{Int64}}
    
    # total crews suppressing each fire
    supp_fire::Array{Vector{Int64}}
    
    # start constraints
    start::Array{Vector{Int64}}
    
    # assignments out of region
    out_of_region::Array{Vector{Int64}}
    
end 

In [13]:
function define_network_constraint_data(arcs)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # get number of arcs
    n_arcs = length(arcs[:, 1])
      
    ## flow balance ##
    
    # initialize arrays of vectors for flow balance
    f_out = Array{Vector{Int64}}(undef, C, G, T, 2)
    f_in = Array{Vector{Int64}}(undef, C, G, T, 2)
    b_out = Array{Vector{Int64}}(undef, C, T, 2)
    b_in = Array{Vector{Int64}}(undef, C, T, 2)
    start = Array{Vector{Int64}}(undef, C)
    out_of_region = Array{Vector{Int64}}(undef, C, T+1)
    
    # for each crew
    for crew in 1:C
        
        # get indices of this crew's arcs only
        crew_ixs = [i for i in 1:n_arcs if arcs[i, 1] == crew]
        
        # get time 0 indices
        start[crew] = [i for i in crew_ixs if arcs[i, 6] == 0]
        
        # for each time period (including start)
        for tm in 0:T
        
            # get indices for out of region assignments
            out_of_region[crew, tm+1] = [i for i in crew_ixs if
                                           (arcs[i, 6] == tm) &
                                           (arcs[i, 10] > 0)
                                        ]
        end
        
        # for each time period
        for tm in 1:T
            
            # for each rest state
            for rest in 1:2
                
                # get arcs leaving crew base at this time with this rest
                b_out[crew, tm, rest] = [i for i in crew_ixs if
                                         (arcs[i, 2] == BASE_CODE) &
                                         (arcs[i, 6] == tm) &
                                         (arcs[i, 8] == rest-1)
                                        ]
                
                # get arcs entering crew base at this time with this rest
                b_in[crew, tm, rest] = [i for i in crew_ixs if
                                        (arcs[i, 4] == BASE_CODE) &
                                        (arcs[i, 7] == tm) &
                                        (arcs[i, 9] == rest-1)
                                       ]
                # for each fire
                for fire in 1:G
                    
                    # get arcs where this crew leaves this fire at this time
                    # with this rest state
                    f_out[crew, fire, tm, rest] = [i for i in crew_ixs if
                                                   (arcs[i, 2] == FIRE_CODE) &
                                                   (arcs[i, 3] == fire) &
                                                   (arcs[i, 6] == tm) &
                                                   (arcs[i, 8] == rest-1)
                                                   ]
                    
                    # get arcs where this crew enters this fire at this time
                    # with this rest state
                    f_in[crew, fire, tm, rest] = [i for i in crew_ixs if
                                                  (arcs[i, 4] == FIRE_CODE) &
                                                  (arcs[i, 5] == fire) &
                                                  (arcs[i, 7] == tm) &
                                                  (arcs[i, 9] == rest-1)
                                                  ]
                end
            end
        end
    end
    
    ## linking constraints ##
    linking = Array{Vector{Int64}}(undef, G, T)
    for fire in 1:G
        for tm in 1:T
            
            # we count the crew as working *where they arrived* during this timestep
            linking[fire, tm] = [i for i in 1:n_arcs if (arcs[i, 4] == FIRE_CODE) &
                                                        (arcs[i, 5] == fire) &
                                                        (arcs[i, 7] == tm)]
        end
    end
    
    
    return KeyArcIndices(f_out, f_in, b_out, b_in, linking, start, out_of_region)
end

define_network_constraint_data (generic function with 1 method)

In [14]:
function get_route_stats(arc_ixs_used, arcs, costs)
    
    # get total cost
    route_cost = sum(costs[arc_ixs_used])
    
    # initialize fires fought matrix
    fires_fought =  falses(NUM_FIRES, NUM_TIME_PERIODS)
    
    # initialize out of region matrix
    out_of_region = falses(NUM_TIME_PERIODS + 1)
    
    # for each arc used
    for ix in arc_ixs_used
        arc = arcs[ix, :]
        
        # update fires_fought
        if (arc[4] == FIRE_CODE) & (arc[7] <= NUM_TIME_PERIODS)
            @assert ~fires_fought[arc[5], arc[7]] "Visited fire twice at same time"
            fires_fought[arc[5], arc[7]] = true
        end
        
        # update out_of_region
        if arc[10] > 0
            @assert ~out_of_region[arc[6] + 1] "Left region twice at same time"
            out_of_region[arc[6] + 1] = true
        end
    end
    
    return route_cost, fires_fought, out_of_region
end

get_route_stats (generic function with 1 method)

In [15]:
function initialize_route_data(max_routes)
    
    return RouteData(zeros(NUM_CREWS), Matrix{Float64}(undef, NUM_CREWS, max_routes),
                     BitArray(undef, NUM_CREWS, max_routes, NUM_FIRES, NUM_TIME_PERIODS) .> 2,
                     BitArray(undef, NUM_CREWS, max_routes, NUM_TIME_PERIODS + 1) .> 2)
end

initialize_route_data (generic function with 1 method)

In [16]:
function update_available_routes(crew, route_ixs, arcs, costs, route_data)
    
    # get the required information from the arcs used
    route_cost, fires_fought, out_of_region = get_route_stats(route_ixs, arcs, costs)
    
    ## store this information to the route_data ##
    
    # add 1 to number of routes for this crew, store the index
    route_data.routes_per_crew[crew] += 1
    ix = route_data.routes_per_crew[crew]
    
    # append the route cost
    route_data.route_costs[crew, ix] = route_cost
    
    # append the fires fought
    route_data.fires_fought[crew, ix, :, :] = fires_fought
    
    # append the out-of-region assignments
    route_data.out_of_reg[crew, ix, :] = out_of_region
    
    return 1

end

update_available_routes (generic function with 1 method)

In [587]:
function get_supp_plan_stats(var_p, var_d, beta, tolerance=0.0001)
    
    # get the cost based on the perimeter progression
    cost = beta * (sum(value.(var_p)) - value(var_p[1])/2 - value(var_p[NUM_TIME_PERIODS+1]/2))
    
    # get the number of crews present each time period from line constructed
    crew_vector = value.(var_d)
    int_crew_vector = convert.(Int64, round.(crew_vector))
    @assert maximum(abs.(crew_vector - int_crew_vector)) < tolerance "Not an integer plan"
    
    return cost, int_crew_vector

end

get_supp_plan_stats (generic function with 3 methods)

In [18]:
function initialize_supp_plan_data(max_supp_plans)
    
    return SuppressionPlanData(zeros(NUM_FIRES), 
                               Matrix{Float64}(undef, NUM_FIRES, max_supp_plans),
                               zeros(Int8, (NUM_FIRES, max_supp_plans, NUM_TIME_PERIODS))
                              )
end

initialize_supp_plan_data (generic function with 1 method)

In [588]:
function update_available_supp_plans(fire, p, d, beta, plan_data)
    
    # get the required information from the model decision variables
    cost, crew_vector = get_supp_plan_stats(p, d, beta)
    
    ## store this information to the suppression plan data ##
    
    # add 1 to number of plans for this fire, store the index
    plan_data.plans_per_fire[fire] += 1
    ix = plan_data.plans_per_fire[fire]
    
    # append the route cost
    plan_data.plan_costs[fire, ix] = cost
    
    # append the fires fought
    plan_data.crews_present[fire, ix, :] = crew_vector
    
    return 1

end

update_available_supp_plans (generic function with 2 methods)

In [590]:
function full_formulation(integer_routes, region_data, constraint_data, rotation_order, 
                          costs, progs, perims, beta, gamma, verbose=false)
    
    # get number of arcs
    n_arcs = length(costs)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    regs = region_data.crew_regions
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    
    if ~verbose
        set_optimizer_attribute(m, "OutputFlag", 0)
    end

    # fire suppression plan section
    @variable(m, p[g=1:G, t=1:T+1] >= 0)
    @variable(m, l[g=1:G, t=1:T])
    @variable(m, d[g=1:G, t=1:T] >= 0)
    @constraint(m, perim_growth[g=1:G, t=1:T], p[g, t+1] >= progs[g, t] * 
                                                           (p[g, t] - l[g, t] / 2) - l[g, t] / 2)
    @constraint(m, perim_start[g=1:G], p[g, 1] == perims[g])

    # routing plan section
    if integer_routes
        @variable(m, z[1:n_arcs] >= 0, Int)
    else
        @variable(m, z[1:n_arcs] >= 0)
    end
    
    @variable(m, q[1:C, 0:T] >= 0, Int)
    
    # build out_of_region constraints
    @constraint(m, out_of_region[c=1:C, t=0:T],
    
        # out of region penalty is at least
        q[c, t] >=
        
            # this crew's cumulative rotations
            sum(z[i] for t_0=0:t, i in constraint_data.out_of_region[c, t_0+1]) 
        
        - 
        
            # average cumulative rotations among all crews in same region
            sum(z[i] for c_0 in keys(rotation_order[regs[c]]), t_0=0:t, 
                i in constraint_data.out_of_region[c_0, t_0+1]) /
            length(keys(rotation_order[regs[c]]))
        
        -
        
            # normalizing factor for specific crew rotation order
            (1 - rotation_order[regs[c]][c] / length(keys(rotation_order[regs[c]])))
        
        -
            # big-M for if crew goes not leave region at this time
            T * (1 - sum(z[i] for i in constraint_data.out_of_region[c, t+1]))
        
    )


    @constraint(m, fire_flow[c=1:C, g=1:G, t=1:T, rest=1:2],

            sum(z[constraint_data.f_out[c, g, t, rest]]) ==
            sum(z[constraint_data.f_in[c, g, t, rest]])
    
    )
    
    @constraint(m, base_flow[c=1:C, t=1:T, rest=1:2],

            sum(z[constraint_data.b_out[c, t, rest]]) ==
            sum(z[constraint_data.b_in[c, t, rest]])
    
    )


    @constraint(m, linking[g=1:G, t=1:T],

        sum(z[constraint_data.supp_fire[g, t]]) >= d[g, t] 
    )
    
    @constraint(m, line_building[g=1:G, t=1:T], l[g, t] <= LINE_PER_CREW * d[g, t])

    # build start constraint
    @constraint(m, start[c=1:C], 

        sum(z[constraint_data.start[c]]) == 1
    )
    
    
    

    @objective(m, Min, 
        beta * (sum(p) - sum(p[1:G, 1])/2 - sum(p[1:G, T+1])/2) + 
        sum(z .* costs) + sum(q) * gamma
    )
    
    return m, p, d, z, q, out_of_region
    
end

full_formulation (generic function with 2 methods)

In [21]:
function load_data(path)
    
    # get distance from fire f to fire g 
    fire_dists =  readdlm(path * "/fire_distances.csv", ',')

    # get distance from base c to fire g (NUM_CREWS-by-NUM_FIRES)
    base_fire_dists =  readdlm(path * "/base_fire_distances.csv", ',')

    # initialize travel times (number of periods) from fire f to fire g
    tau = convert(Array{Int}, ones(size(fire_dists)))

    # initialize number of periods to travel from base c to fire g (NUM_CREWS-by-NUM_FIRES)
    tau_base_to_fire = convert(Array{Int}, ones((size(base_fire_dists))))

    # read intial crew statuses (location, period by which they must rest)
    # (-1 in current_fire means crew is currently at base)
    # (rested_periods is the amount of time crew has been at base, relevant for completing rest)
    crew_starts = CSV.read(path * "/sample_crew_starts.csv", DataFrame)
    rest_by = crew_starts[!, "rest_by"]
    current_fire = crew_starts[!, "current_fire"]
    rested_periods = crew_starts[!, "rested_periods"]


    return (GlobalData(fire_dists, base_fire_dists, tau, tau_base_to_fire), 
            CrewStatus(rest_by, current_fire, rested_periods))
end

load_data (generic function with 1 method)

In [318]:
function master_problem(route_data, supp_plan_data, region_data, rotation_order, gamma)
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)
    
    regs = region_data.crew_regions
    
    # decision variables
    @variable(m, route[c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c]] >= 0)
    @variable(m, plan[g=1:NUM_FIRES, p=1:supp_plan_data.plans_per_fire[g]] >= 0)
    @variable(m, q[c=1:NUM_CREWS, t=0:NUM_TIME_PERIODS] >= 0)
    
    # constraints that you must choose a plan per crew and per fire
    @constraint(m, route_per_crew[c=1:NUM_CREWS], 
                sum(route[c, r] for r=1:route_data.routes_per_crew[c]) == 1)
    @constraint(m, plan_per_fire[g=1:NUM_FIRES], 
                sum(plan[g, p] for p=1:supp_plan_data.plans_per_fire[g]) >= 1)
    
    # linking constraint
    @constraint(m, linking[g=1:NUM_FIRES, t=1:NUM_TIME_PERIODS],
                    
                    # crews at fire
                    sum(route[c, r] * route_data.fires_fought[c, r, g, t] 
                        for c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c]) 
        
                    >=
        
                    # crews suppressing
                    sum(plan[g, p] * supp_plan_data.crews_present[g, p, t] 
                        for p=1:supp_plan_data.plans_per_fire[g]) 
        
                )
    
    # out_of_region constraint
    @constraint(m, out_of_region[c=1:NUM_CREWS, t=0:NUM_TIME_PERIODS],
    
        # out of region penalty is at least
        q[c, t] >=
        
            # this crew's cumulative rotations
            sum(route[c, r] * route_data.out_of_reg[c, r, t_0 + 1] 
            for r=1:route_data.routes_per_crew[c], t_0=0:t)
        
        - 
        
            # average cumulative rotations among all crews in same region
            sum(route[c_0, r] * route_data.out_of_reg[c_0, r, t_0 + 1] 
                for c_0 in keys(rotation_order[regs[c]]), r=1:route_data.routes_per_crew[c_0],
                t_0=0:t) /
            length(keys(rotation_order[regs[c]]))
        
        -
        
            # normalizing factor for specific crew rotation order
            (1 - rotation_order[regs[c]][c] / length(keys(rotation_order[regs[c]])))
        
        -
            # big-M for if crew goes not leave region at this time
            NUM_TIME_PERIODS * (1 - sum(route[c, r] * route_data.out_of_reg[c, r, t+1]
                                        for r=1:route_data.routes_per_crew[c])
                               )
        
    )
    
    @objective(m, Min, 
        
                  # route costs
                  sum(route[c, r] * route_data.route_costs[c, r] 
                        for c=1:NUM_CREWS, r=1:route_data.routes_per_crew[c])
        
                  +
                     
                  # suppression plan costs
                  sum(plan[g, p] * supp_plan_data.plan_costs[g, p] 
                     for g=1:NUM_FIRES, p=1:supp_plan_data.plans_per_fire[g]) 
        
                  +
        
                  # rotational queueing violations cost
                  sum(q) * gamma
               )
    
    return Dict("m" => m, "q" => q, "sigma" => route_per_crew, "pi" => plan_per_fire, 
                "rho" => linking, "eta" => out_of_region, "route" => route, "plan" => plan)
end 

master_problem (generic function with 1 method)

In [60]:
function init_route_subproblem(crew_ixs, crew, constraint_data, integer_routes=false)
    
    # shorten some global variable names
    C = NUM_CREWS
    G = NUM_FIRES
    T = NUM_TIME_PERIODS
    
    # intialize model
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # routing plan section
    if integer_routes
        @variable(m, z[crew_ixs] >= 0, Int)
    else
        @variable(m, z[crew_ixs] >= 0)
    end


    @constraint(m, fire_flow[g=1:G, t=1:T, rest=1:2],

            sum(z[constraint_data.f_out[crew, g, t, rest]]) ==
            sum(z[constraint_data.f_in[crew, g, t, rest]])
    
    )
    
    @constraint(m, base_flow[t=1:T, rest=1:2],

            sum(z[constraint_data.b_out[crew, t, rest]]) ==
            sum(z[constraint_data.b_in[crew, t, rest]])
    
    )

    # build start constraint
    @constraint(m, start, 

        sum(z[constraint_data.start[crew]]) == 1
    )
    
    return Dict("m" => m, "z" => z, "ff" => fire_flow)
    
end

init_route_subproblem (generic function with 2 methods)

In [564]:
function init_suppression_plan_subproblem(progs, perims, fire, beta)
    
    T = NUM_TIME_PERIODS
    
    m = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(m, "OutputFlag", 0)

    # fire suppression plan section
    @variable(m, p[t=1:T+1] >= 0)
    @variable(m, l[t=1:T] >= 0)
    @variable(m, NUM_CREWS >= d[t=1:T] >= 0, Int)
    @constraint(m, suppression_per_crew[t=1:T], l[t] <= d[t] * LINE_PER_CREW)
    @constraint(m, perim_growth[t=1:T], p[t+1] >= progs[fire, t] * (p[t] - l[t] / 2) - l[t] / 2)
    @constraint(m, perim_start, p[1] == perims[fire])
    
#    
    
    return Dict("m" => m, "p" => p, "d" => d, "beta" => beta)
end

init_suppression_plan_subproblem (generic function with 2 methods)

In [852]:
in_path = "data/processed"

# get inital fire perimeters and no-suppression progression parameters
M = readdlm(in_path * "/sample_growth_patterns.csv", ',')
start_perims = M[:, 1]
progressions = M[:, 2:15]

NUM_TIME_PERIODS = size(M)[2] - 1 
NUM_FIRES = size(M)[1]      

g_data, crew_status = load_data(in_path)
r_data = RegionData([1, 1, 1, 1, 1, 2, 2, 2, 2, 2], [1, 1, 2, 2])
rotation_order = get_rotation_orders(r_data.crew_regions)
A = generate_arcs(g_data, r_data, crew_status);

rest_pen = get_rest_penalties(crew_status.rest_by, 99999, positive)
cost_params = Dict("cost_per_mile"=> 1, "rest_violation" => rest_pen, "fight_fire" => ALPHA)
arc_costs = get_arc_costs(g_data, A, cost_params)

c_data = define_network_constraint_data(A);

In [853]:
route_sps = []
for crew in 1:NUM_CREWS
    ixs = [i for i in 1:length(A[:, 1]) if A[i, 1] == crew]
    d = init_route_subproblem(ixs, crew, c_data)
    d["arc_ixs"] = ixs
    push!(route_sps, d)
end

plan_sps = []
for fire in 1:NUM_FIRES
    d = init_suppression_plan_subproblem(progressions, start_perims, fire, BETA)
    push!(plan_sps, d)
end

In [854]:
m, p, d, z, q, oor = full_formulation(true, r_data, c_data, rotation_order, arc_costs, 
                              progressions, start_perims, BETA, 0)
optimize!(m);

In [1112]:
routes = initialize_route_data(1000)
suppression_plans = initialize_supp_plan_data(1000)


for crew in 1:NUM_CREWS
    # get the crew's subproblem instance
    crew_sp = route_sps[crew]
    m = crew_sp["m"]
    z = crew_sp["z"]
    crew_ixs = crew_sp["arc_ixs"]
    
    # set objective in light of dual variables
    @objective(m, Min, sum(z[ix] * (arc_costs[ix]) for ix in crew_ixs))
        
    # optimize
    optimize!(m)
    
    
    # update crew plans
    crew_arcs = [i for i in crew_ixs if (value(z[i]) > 0.5)]
    update_available_routes(crew, crew_arcs, A, arc_costs, routes)
    
end

for fire in 1:NUM_FIRES
    
    plan_sp = plan_sps[fire]
    m = plan_sp["m"]
    p = plan_sp["p"]
    d = plan_sp["d"]
    beta = plan_sp["beta"]
    
    @objective(m, Min, sum(d))
    optimize!(m)
    update_available_supp_plans(fire, p, d, beta, suppression_plans)
        
end
    

In [1193]:
mp = master_problem(routes, suppression_plans, r_data, rotation_order, 10000)
optimize!(mp["m"])
sigma = dual.(mp["sigma"])
rho = dual.(mp["rho"])
eta = dual.(mp["eta"])
pie = dual.(mp["pi"]) # lol can't overwrite "pi" in Julia

d = Dict("out_of_region_dual" => eta, "region_data"=> r_data, "rotation_order" => rotation_order, "linking_dual" => rho)
local_arc_costs = get_arc_costs(g_data, A, d)

for fire in 1:NUM_FIRES
    
    plan_sp = plan_sps[fire]
    m = plan_sp["m"]
    p = plan_sp["p"]
    d = plan_sp["d"]
    beta = plan_sp["beta"]
    
    @objective(m, Min, beta * (sum(p) - p[1]/2 - p[NUM_TIME_PERIODS + 1]/2) + sum(d .* rho[fire, :]) + 0.0001 * sum(d))
    optimize!(m)
    if objective_value(m) < pie[fire]
        
        update_available_supp_plans(fire, p, d, beta, suppression_plans)
        
    end
end
    
    
for crew in 1:NUM_CREWS
    
    # get the crew's subproblem instance
    crew_sp = route_sps[crew]
    m = crew_sp["m"]
    z = crew_sp["z"]
    crew_ixs = crew_sp["arc_ixs"]
    
    # set objective in light of dual variables
    @objective(m, Min, sum(z[ix] * (local_arc_costs[ix] + arc_costs[ix]) for ix in crew_ixs))
        
    # optimize
    optimize!(m)
    
    if objective_value(m) < sigma[crew]
    
        # update crew plans
        crew_arcs = [i for i in crew_ixs if (value(z[i]) > 0.5)]
        update_available_routes(crew, crew_arcs, A, arc_costs, routes)
        
    end
    
end   

routes.routes_per_crew, suppression_plans.plans_per_fire, objective_value(mp["m"])

([40, 60, 54, 37, 51, 55, 35, 36, 72, 71], [49, 38, 13], 1.1512450362687232e6)

In [960]:
routes.routes_per_crew, suppression_plans.plans_per_fire, objective_value(mp["m"])

([47, 49, 51, 43, 54, 61, 40, 44, 72, 52], [40, 37, 13], 1.1510952749095825e6)

In [1209]:
a = zeros(NUM_FIRES, NUM_TIME_PERIODS)
for g=1:NUM_FIRES
    for t=1:NUM_TIME_PERIODS
        for c=1:NUM_CREWS
            for r=1:routes.routes_per_crew[c]
                a[g, t] += value(mp["route"][c, r]) * routes.fires_fought[c, r, g, t] 
            end
        end
    end
end

LoadError: KeyError: key (2, 60) not found

In [864]:
sum(routes.fires_fought[:, 1, :, :])

0

In [1111]:
eta

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(10)
    Dimension 2, 0:14
And data, a 10×15 Matrix{Float64}:
 0.0  0.0  0.0    0.0    0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0    0.0    0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  501.153  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [114]:
routes.out_of_reg[:, 1, :]

10×15 BitMatrix:
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  1  0  0  0  1  0  1  0
 0  0  1  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  1  0  0

In [42]:
crew_1_arcs = [i for i=1:length(A[:, 1]) if (value(z[i]) > 0.5) & (A[i, 1] == 1)];
cost, ff, oor = get_route_stats(crew_1_arcs, A, arc_costs);

In [170]:
value.(l) / LINE_PER_CREW

3×14 Matrix{Float64}:
 0.0  0.0  0.0  0.0  3.0  7.0  8.0         8.0  8.0  5.0  3.0  3.0  4.0  4.0
 0.0  7.0  9.0  9.0  4.0  0.0  4.11765e-7  0.0  0.0  1.0  1.0  2.0  0.0  3.0
 9.0  2.0  0.0  0.0  0.0  0.0  0.0         0.0  0.0  0.0  1.0  1.0  1.0  1.0

In [34]:
value.(l) / LINE_PER_CREW

3×14 Matrix{Float64}:
 0.0  0.0  0.0  0.0  3.0  7.0  8.0  8.0  8.0  6.0  2.0  2.0  2.0  6.0
 0.0  7.0  9.0  9.0  4.0  0.0  0.0  0.0  0.0  0.0  2.0  1.0  2.0  0.0
 9.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0

In [33]:
(sum(get_arc_costs(global_data, A2, Dict("cost_per_mile"=>10, "rest_violation" => ones(NUM_CREWS, NUM_TIME_PERIODS)))) - 
    sum(get_arc_costs(global_data, A2, Dict("cost_per_mile"=>10))))

2380.0

In [9]:
function get_out_of_region_stats(region, arcs_used, region_data)
    """
    """
    
    # restrict to the arcs that exited the given region
    out_of_region_ixs = [i for i in length(arcs_used[:, 1]) if arcs_used[i, 10] == region]
    out_of_region_arcs = arcs_used[out_of_region_ixs, :]
    
    # get the crews associated with this region
    crews = [i for i in 1:length(region_data.crew_regions) if region_data.crew_regions[i] == region]
    
    # initialize output array of indicator variables for crews exiting region
    out_array = zeros(length(crews), NUM_TIME_PERIODS)
    
    # for each crew in the region
    for i in 1:length(crews)
        
        # restrict to the arcs involving this crew
        crew_ixs = [j for j in length(out_of_region_arcs[:, 1]) if arcs_used[j, 1] == crews[i]]
        crew_arcs = out_of_region_arcs[crew_ixs, :]
        
        # get the times of rotation
        rotation_times = [t in crew_arcs[:, 6] ? 1 : 0 for t in 1:NUM_TIME_PERIODS]
        
        # update output for crew
        out_array[i, :] = rotation_times
        
    end
    
    return out_array
end

LoadError: syntax: incomplete: "function" at In[9]:1 requires end